Hi there! In this notebook we are creating responses for our chatbot based on the trained model. We will
- load our model, important data structures and the datafile containing the intents of our chatbot
- classify sentence into difrrent categoriees
- creating a response for our chatbot

In [1]:
# things we need for language processing
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# we will need the tensorflow and numpy library 
import numpy as np
import tensorflow as tf

# for creating some randomness in the chatbot responses
import random

In [2]:
# restore all of your data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
corpus = data['corpus']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import the data file with the intents for your chatbot
import json
with open('datafile.json') as json_data:
    intents = json.load(json_data)

In [3]:
# preprocess a sentence 
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, corpus):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag_of_words = [0]*len(corpus)  
    for s in sentence_words:
        for i,w in enumerate(corpus):
            if w == s: 
                bag_of_words[i] = 1

    return(np.array(bag_of_words))

In [4]:
# load our trained model
model = tf.keras.models.load_model('my_model')

In [5]:
# classify a sentence into categories defined by 'classes' array
def classify(sentence):
    # generate probabilities from the model
    results = model.predict(bow(sentence, corpus).reshape(1,-1))[0]
    results = [[i,r] for i,r in enumerate(results)]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

# We add 'context' to the dialog: if a category contains a 'context_filter', a response from this category 
# can only be activated if another category with a 'context_set' was applied first. 
# E.g: A response from the category 'topic' can only be activated if the category 'courses' was applied before.
context = set()

# Define a threshold of confidence, if a category get's a probability above this threshold
# the chatbot gives an appropriate response, otherwise he will reply that he doesn't understand.
CONFIDENCE_LEVEL = 0.5
do_not_understand = ["I don't understand..", "Rephrase please..", "Uff, I don't get you.."]

# create a response from the input sentence
def response(sentence, show_details=False):
    
    global context
    
    results = classify(sentence)
    if show_details:
        print('results:', results)

    # if we have a classification then find the matching intent tag, otherwise reply that it doesn't understand. 
    if results and (results[0][1] > CONFIDENCE_LEVEL):
        # loop as long as there are matches to process
        while results:
            for intent in intents['intents']:
                # find a tag matching the first result
                if intent['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in intent:
                        context.clear()
                        context.add(intent['context_set'])
                        if show_details:
                            print ('context:', context)

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in intent or ('context_filter' in intent and intent['context_filter'] in context):
                        # a random response from the intent
                        return random.choice(intent['responses'])

            results.pop(0)
        return random.choice(do_not_understand) 
    else:
        return random.choice(do_not_understand) 

Nice! Now you have implemented the response for your chatbot. The only thing left to do is to start chatting. ;-)